In [114]:
# Dependencies
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import *
from config import username, password

In [116]:
dfPop2000 = pd.DataFrame
csvFile = 'Resources/population_by_zip_2000.csv'
dfPop2000 = pd.read_csv(csvFile,delimiter=',',encoding='utf-8',low_memory=False,dtype=str)
dfPop2010 = pd.DataFrame
csvFile = 'Resources/population_by_zip_2010.csv'
dfPop2010 = pd.read_csv(csvFile,delimiter=',',encoding='utf-8',low_memory=False,dtype=str)
dfBars = pd.DataFrame
csvFile = 'Resources/8260_1.csv'

dfBars = pd.read_csv(csvFile,delimiter=',',encoding='utf-8',low_memory=False,dtype=str)
dfBars.dropna(subset=['postalCode'],inplace=True) # Drop rows with no zip code listed
dfBars['postalCode'] = dfBars['postalCode'].str.zfill(5) # Prepend '0' if zip code less than 5 digits long
dfBars['postalCode'] = dfBars['postalCode'].str.slice(start=0,stop=5) # Keep only first 5 digits of zip code
bars_columns = ['postalCode','name']
bars_clean = dfBars[bars_columns].copy()
dfBars=bars_clean.rename(columns={'postalCode':'zipcode'})

ZipBars = dfBars.groupby(['zipcode']).count()
ZipBars.sort_values('name',ascending=False,inplace=True)
ZipBars.rename(columns={'name':'bar_count'},inplace=True)

dfPop2010.dropna(subset=['zipcode'],inplace=True)
dfPop2010['zipcode'] = dfPop2010['zipcode'].str.zfill(5)
dfPop2010['zipcode'] = dfPop2010['zipcode'].str.slice(start=0,stop=5)

dfPop2000.dropna(subset=['zipcode'],inplace=True)
dfPop2000['zipcode'] = dfPop2000['zipcode'].str.zfill(5)
dfPop2000['zipcode'] = dfPop2000['zipcode'].str.slice(start=0,stop=5)


,minimum_age,maximum_age,gender,population,zipcode,geo_id
0,10,14,female,75,39769,8600000US39769
1,85,NaN,female,130,44047,8600000US44047
2,30,34,female,11,42436,8600000US42436
3,22,24,male,22,99685,8600000US99685
4,67,69,male,69,04239,8600000US04239


In [117]:
# Create connection to Bars_db
rds_connection_string = username + ':' + password + '@127.0.0.1/Bars_db'
engine = create_engine(f'mysql://{rds_connection_string}')
ZipBars.to_sql(name='bars_per_zipcode', con=engine, if_exists='replace', index=True, index_label='zipcode',dtype={'zipcode':String(5)})
engine.table_names()

,population,minimum_age,maximum_age,gender,zipcode,geo_id
0,50,30,34,female,61747,8600000US61747
1,5,85,NaN,male,64120,8600000US64120
2,1389,30,34,male,95117,8600000US95117
3,231,60,61,female,74074,8600000US74074
4,56,0,4,female,58042,8600000US58042


In [ ]:
pd.read_sql_query('select * from bars_per_zipcode', con=engine).head()